In [108]:
# install.packages("xgboost")
# install.packages("Metrics")
# install.packages("FactoMineR")

In [109]:
library(xgboost)
library(Metrics)
library(tseries)
library(FactoMineR)
library(tidyverse) # For data manipulation and visualization
library(forecast)

In [110]:
data_temp = read.csv("forecast_competition_training.csv")

temp_y = data_temp[2:500, 1]

data = data_temp[1:499, ]
data_pred = data_temp[500, ]

data$y = temp_y

data_train = data[1:470, ]
data_test = data[471:499, ]

data

,TARGET,IMBFM,HPDYO,KRZRU,QHOTV,GODCT,NIYPB,DWAMN,TBBPE,XUPPP,...,QHTJG,DHSKT,UNCHW,ITMOC,HNATR,DSMIG,PHBYE,NZHPQ,LMOBP,y
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.13317406,-0.45563917,0.41917864,-0.28206491,0.28355820,-0.09035003,-1.21280954,-0.410329108,-1.9672935,0.250595075,...,1.3230764,0.27686594,2.16923304,-0.67111873,-0.78128215,2.56478662,-0.587409247,-2.34389451,0.39126225,1.27200611
2,1.27200611,0.43593406,0.50953265,-1.44664880,-1.15731920,1.80887452,-0.08384729,0.751965184,-0.6942029,-0.473482606,...,0.9390097,-0.58134069,-0.41611020,1.49020632,-0.14141558,-0.97144818,-1.435006848,0.03938233,-1.20273639,1.34196938
3,1.34196938,0.52793899,-1.73709488,0.40749478,0.89149141,-0.72539831,1.48683632,1.348854817,-0.9235301,-0.816597929,...,1.1369724,1.29474441,0.25547520,-0.72513235,-0.55543958,-0.04418441,-0.125330668,3.79373990,0.18452093,-0.16247284
4,-0.16247284,-0.08453185,0.39550499,-0.31482236,-0.30459798,-0.39525143,-0.45536039,0.475327024,0.7959494,1.402792793,...,-0.7108817,1.03186216,-2.92439455,0.97866615,2.52151500,-0.64466640,-2.415052232,-0.85084024,-1.55733759,0.19871226
5,0.19871226,0.41014607,2.52377365,0.14172137,-0.19716104,0.56951431,-0.66841141,0.593282073,-0.1450747,1.860533207,...,-0.9780717,0.78734959,-0.09875043,-0.14935249,0.42166991,-0.96162820,-2.282784536,-2.08115119,-1.57941991,-1.08215918
6,-1.08215918,-3.04692715,1.21305384,0.23895999,-1.02380767,-0.54971234,0.42391073,-0.058403927,0.4887470,-0.755036521,...,-0.3956842,0.30456031,-0.94670941,-1.63409460,-0.83060580,0.81823952,-0.464791542,-2.98164741,-0.40692582,0.03243700
7,0.03243700,0.53028564,0.65710353,-1.87523483,0.86685096,-0.79408219,1.79913876,-1.532058754,-0.3333583,-0.621027894,...,1.7302744,-0.62838076,-1.53175256,-3.27875602,-1.69627744,-0.71481478,-0.004198675,0.71362952,-1.77171380,-1.38363510
8,-1.38363510,-0.69075633,1.24120862,0.31787774,0.89816857,0.47477943,0.04392141,-0.328330479,-0.5036497,-1.633543693,...,2.0753155,1.25182619,0.48160558,2.88157117,0.35795532,-0.75600144,0.401299358,-1.42882132,1.85976264,-2.80954522
9,-2.80954522,-1.65594844,3.18175447,-0.90722885,-4.00986189,-1.09723300,1.67552038,-2.751592976,1.1359111,-1.371370024,...,-0.9041685,-0.19316947,-0.25032717,0.13157157,-0.37842476,0.93875405,0.491440417,-1.21833050,1.18235868,-1.89625550


In [111]:
y_train = data_train[, 51]
X_train = data_train[, -51]
y_test = data_test[, 51]
X_test = data_test[, -51]

In [112]:
pca_result <- PCA(X_train, scale.unit = TRUE, graph = FALSE, ncp = 30)

In [113]:
explained_variance <- pca_result$eig[,2] # Extract explained variance percentages
cum_explained_variance <- cumsum(explained_variance) # Cumulative variance explained
num_components <- which(cum_explained_variance > 85)[1] # Number of components explaining > 85% variance

print(paste("Total principal components:", ncol(pca_result$ind$coord)))

# Print the cumulative explained variance
print(cum_explained_variance)
print(num_components)
print(pca_result$ind$coord)

# Get scores of the selected principal components
scores <- pca_result$ind$coord[, 1:num_components]

[1] "Total principal components: 30"
   comp 1    comp 2    comp 3    comp 4    comp 5    comp 6    comp 7    comp 8 
 31.36927  51.19398  54.57605  57.26068  59.79403  62.20403  64.45837  66.50145 
   comp 9   comp 10   comp 11   comp 12   comp 13   comp 14   comp 15   comp 16 
 68.46351  70.36294  72.07656  73.72830  75.34036  76.91379  78.41205  79.83405 
  comp 17   comp 18   comp 19   comp 20   comp 21   comp 22   comp 23   comp 24 
 81.18469  82.48297  83.68265  84.78101  85.82063  86.81127  87.73609  88.62809 
  comp 25   comp 26   comp 27   comp 28   comp 29   comp 30   comp 31   comp 32 
 89.44483  90.22468  90.97904  91.72485  92.42919  93.10244  93.72207  94.26238 
  comp 33   comp 34   comp 35   comp 36   comp 37   comp 38   comp 39   comp 40 
 94.77012  95.24132  95.70089  96.12758  96.51500  96.89736  97.26064  97.59647 
  comp 41   comp 42   comp 43   comp 44   comp 45   comp 46   comp 47   comp 48 
 97.92516  98.21821  98.49508  98.76279  99.00719  99.24053  99.46034  9

In [114]:
model <- lm(y_train ~ scores)

In [120]:
actual_new_observation_scores <- predict(pca_result, newdata = X_test)$ind$coord[, 1:num_components]

# Predict the target variable using the new PCA scores
actual_predicted_values <- predict(model, newdata = as.data.frame(actual_new_observation_scores))

# Print the predicted values for the actual new data
print(actual_predicted_values)

           1            2            3            4            5            6 
 0.264274508  0.346008665  0.518786705  0.417441322  0.197123713 -0.694472482 
           7            8            9           10           11           12 
-0.384258237 -1.122025802 -1.311856497 -1.018633204 -0.488470456  0.926369685 
          13           14           15           16           17           18 
 1.184269107  1.102350230  0.602895503  0.785195337  0.930335266  0.563837191 
          19           20           21           22           23           24 
-0.334076471  0.191140183  0.702904189  1.036256093  0.952025047  0.784650116 
          25           26           27           28           29           30 
 0.088423888 -0.153959315 -0.857563745 -0.746758742 -0.294473289  0.524760288 
          31           32           33           34           35           36 
-0.349160078  0.491270074 -0.497890289 -0.725201340 -0.336020263  0.378078944 
          37           38           39           40 